In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('使用されるデバイス:', device)

使用されるデバイス: cuda


In [2]:
import sys

In [3]:
import torch.utils.data as data

from voc import make_filepath_list, GetBBoxAndLabel, DataTransform, PreprocessVOC2012, multiobject_collate_fn

rootpath = '/home/komuro/Documents/ssdbook/data/VOCdevkit/VOC2012/'
train_img_list, train_anno_list, val_img_list, val_anno_list = make_filepath_list(rootpath)

voc_classes = [
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
    'bus', 'car', 'cat', 'chair', 'cow',
    'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train',
    'tvmonitor'
]

In [4]:
color_mean = (104, 117, 123)
input_size = 300

train_dataset = PreprocessVOC2012(
    train_img_list,
    train_anno_list,
    phase='train',
    transform=DataTransform(input_size, color_mean),
    get_bbox_label=GetBBoxAndLabel(voc_classes)
)

val_dataset = PreprocessVOC2012(
    val_img_list,
    val_anno_list,
    phase='val',
    transform=DataTransform(input_size, color_mean),
    get_bbox_label=GetBBoxAndLabel(voc_classes)
)

In [5]:
batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=multiobject_collate_fn
)
val_dataloader = data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=multiobject_collate_fn
)

dataloaders_dict = {'train': train_dataloader, 'val': val_dataloader}

In [6]:
import torch.nn as nn
import torch.nn.init as init
from ssd import SSD
from torchinfo import summary

ssd_cfg = {
    'classes_num': 21,
    'input_size': 300,
    'dbox_num': [4, 6, 6, 6, 4, 4],
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

net = SSD(phase='train', cfg=ssd_cfg)

vgg_weights = torch.load('/home/komuro/Documents/ssdbook/weights/vgg16_redecedfc.pth')
net.vgg.load_state_dict(vgg_weights)

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

summary(
    net,
    input_size=(batch_size, 3, 300, 300),
    col_names=['input_size', 'output_size', 'num_params']
)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
SSD                                      [32, 3, 300, 300]         [32, 8732, 4]             --
├─ModuleList: 1-3                        --                        --                        (recursive)
│    └─Conv2d: 2-1                       [32, 3, 300, 300]         [32, 64, 300, 300]        1,792
│    └─ReLU: 2-2                         [32, 64, 300, 300]        [32, 64, 300, 300]        --
│    └─Conv2d: 2-3                       [32, 64, 300, 300]        [32, 64, 300, 300]        36,928
│    └─ReLU: 2-4                         [32, 64, 300, 300]        [32, 64, 300, 300]        --
│    └─MaxPool2d: 2-5                    [32, 64, 300, 300]        [32, 64, 150, 150]        --
│    └─Conv2d: 2-6                       [32, 64, 150, 150]        [32, 128, 150, 150]       73,856
│    └─ReLU: 2-7                         [32, 128, 150, 150]       [32, 128, 150, 150]       --
│    └─Conv2d: 

In [7]:
import torch.optim as optim
from ssd import MultiBoxLoss

criterion = MultiBoxLoss(
    jaccard_thresh=0.5,
    neg_pos=3,
    device=device
)

optimizer = optim.SGD(
    net.parameters(),
    lr=1e-3,
    momentum=0.9,
    weight_decay=5e-4
)

In [8]:
import time
import pandas as pd

def train(net, dataloaders_dict, criterion, optimizer, num_epochs):
    net.to(device)
    torch.backends.cudnn.benchmark = True

    iteration = 1
    epoch_train_loss = 0.0
    epoch_val_loss = 0.0
    logs=[]

    for epoch in range(num_epochs):
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('--------------------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('--------------------------')

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                if ((epoch+1) % 10 == 0):
                    net.eval()
                    print('--------------------------')
                    print('validation')
                else:
                    continue

            
            for images, targets in dataloaders_dict[phase]:
                images = images.to(device)

                targets = [ann.to(device) for ann in targets]

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(images)

                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    if phase == 'train':
                        loss.backward()
                        nn.utils.clip_grad_value_(net.parameters(),
                                                  clip_value=2.0)
                        optimizer.step()

                        if (iteration % 10 == 0):
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start

                            print('ステップ( {} )  loss: {:.4f} -- time: {:.4f} sec.'.format(
                                iteration, loss.item(), duration
                            ))
                            t_iter_start = time.time()
                        
                        epoch_train_loss += loss.item()
                        iteration += 1
                    else:
                        epoch_val_loss += loss.item()
        t_epoch_finish = time.time()
        print('--------------------------')
        print('train_loss: {:.4f} - val_loss: {:.4f}'.format(
            epoch_train_loss, epoch_val_loss
        ))

        print('time: {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        log_epoch = {'epoch': epoch+1,
                    'train_loss': epoch_train_loss,
                    'val_loss': epoch_val_loss}
        
        logs.append(log_epoch)
        df = pd.DataFrame(logs)

        df.to_csv('/home/komuro/Documents/ssdbook/epoch_loss.csv')

        epoch_train_loss = 0.0
        epoch_val_loss = 0.0

        if ((epoch+1) % 10 == 0):
            torch.save(
                net.state_dict(),
                '/home/komuro/Documents/ssdbook/weights/ssd_weights' + str(epoch+1) + '.pth'
            )
            print('--saved weights--')







In [9]:
num_epochs = 50

train(net,
      dataloaders_dict,
      criterion,
      optimizer,
      num_epochs=num_epochs)

--------------------------
Epoch 1/50
--------------------------


/home/komuro/Documents/ssdbook/augmentation.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


ステップ( 10 )  loss: 15.8207 -- time: 6.4324 sec.
ステップ( 20 )  loss: 12.6182 -- time: 4.6073 sec.
ステップ( 30 )  loss: 9.8878 -- time: 4.6484 sec.
ステップ( 40 )  loss: 9.4278 -- time: 4.7017 sec.
ステップ( 50 )  loss: 8.5319 -- time: 4.7253 sec.
ステップ( 60 )  loss: 8.4758 -- time: 4.4742 sec.
ステップ( 70 )  loss: 8.3710 -- time: 4.4527 sec.
ステップ( 80 )  loss: 7.7531 -- time: 4.5065 sec.
ステップ( 90 )  loss: 8.0502 -- time: 4.6091 sec.
ステップ( 100 )  loss: 7.8935 -- time: 4.4937 sec.
ステップ( 110 )  loss: 7.8302 -- time: 4.5122 sec.
ステップ( 120 )  loss: 8.2258 -- time: 4.5580 sec.
ステップ( 130 )  loss: 8.2113 -- time: 4.4918 sec.
ステップ( 140 )  loss: 7.8454 -- time: 4.6508 sec.
ステップ( 150 )  loss: 7.7811 -- time: 4.5301 sec.
ステップ( 160 )  loss: 7.2341 -- time: 4.6821 sec.
ステップ( 170 )  loss: 7.6702 -- time: 4.4561 sec.
--------------------------
train_loss: 1665.5530 - val_loss: 0.0000
time: 88.2907 sec.
--------------------------
Epoch 2/50
--------------------------


/home/komuro/Documents/ssdbook/augmentation.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


ステップ( 180 )  loss: 7.7894 -- time: 0.3452 sec.
ステップ( 190 )  loss: 7.3454 -- time: 4.6500 sec.
ステップ( 200 )  loss: 7.1523 -- time: 4.6307 sec.
ステップ( 210 )  loss: 7.3136 -- time: 4.5940 sec.
ステップ( 220 )  loss: 7.0639 -- time: 4.5658 sec.
ステップ( 230 )  loss: 7.0955 -- time: 4.7195 sec.
ステップ( 240 )  loss: 6.9649 -- time: 4.6064 sec.
ステップ( 250 )  loss: 7.4294 -- time: 4.5286 sec.
ステップ( 260 )  loss: 6.8294 -- time: 4.3894 sec.
ステップ( 270 )  loss: 7.5333 -- time: 4.4188 sec.
ステップ( 280 )  loss: 7.4206 -- time: 4.5589 sec.
ステップ( 290 )  loss: 6.7496 -- time: 4.5786 sec.
ステップ( 300 )  loss: 6.5092 -- time: 4.6229 sec.
ステップ( 310 )  loss: 6.9015 -- time: 4.5527 sec.
ステップ( 320 )  loss: 7.3971 -- time: 4.3534 sec.
ステップ( 330 )  loss: 7.3733 -- time: 4.5246 sec.
ステップ( 340 )  loss: 6.7835 -- time: 4.5278 sec.
ステップ( 350 )  loss: 7.1444 -- time: 4.5727 sec.
--------------------------
train_loss: 1271.2633 - val_loss: 0.0000
time: 84.0322 sec.
--------------------------
Epoch 3/50
--------------------------


...

ステップ( 8420 )  loss: 3.8871 -- time: 3.1763 sec.
ステップ( 8430 )  loss: 2.9303 -- time: 4.5552 sec.
ステップ( 8440 )  loss: 3.4763 -- time: 4.5672 sec.
ステップ( 8450 )  loss: 3.3689 -- time: 4.6029 sec.
ステップ( 8460 )  loss: 2.9827 -- time: 4.6354 sec.
ステップ( 8470 )  loss: 3.6471 -- time: 4.6796 sec.
ステップ( 8480 )  loss: 2.8849 -- time: 4.6313 sec.
ステップ( 8490 )  loss: 3.9585 -- time: 4.5058 sec.
ステップ( 8500 )  loss: 3.9513 -- time: 4.6413 sec.
ステップ( 8510 )  loss: 3.5657 -- time: 4.5842 sec.
ステップ( 8520 )  loss: 3.1375 -- time: 4.5903 sec.
ステップ( 8530 )  loss: 3.9060 -- time: 4.5449 sec.
ステップ( 8540 )  loss: 3.4508 -- time: 4.7895 sec.
ステップ( 8550 )  loss: 3.4370 -- time: 4.7173 sec.
ステップ( 8560 )  loss: 4.0058 -- time: 4.6027 sec.
ステップ( 8570 )  loss: 3.7914 -- time: 4.5939 sec.
ステップ( 8580 )  loss: 3.8810 -- time: 4.6725 sec.
ステップ( 8590 )  loss: 3.8545 -- time: 4.6022 sec.
--------------------------
train_loss: 618.2093 - val_loss: 0.0000
time: 85.2271 sec.
--------------------------
Epoch 49/50
-----------

/home/komuro/Documents/ssdbook/augmentation.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


ステップ( 8600 )  loss: 3.5562 -- time: 3.5431 sec.
ステップ( 8610 )  loss: 3.4854 -- time: 4.5997 sec.
ステップ( 8620 )  loss: 3.6750 -- time: 4.5551 sec.
ステップ( 8630 )  loss: 3.6721 -- time: 4.6024 sec.
ステップ( 8640 )  loss: 3.7106 -- time: 4.5798 sec.
ステップ( 8650 )  loss: 3.8157 -- time: 4.7815 sec.
ステップ( 8660 )  loss: 3.5406 -- time: 4.5222 sec.
ステップ( 8670 )  loss: 3.2045 -- time: 4.6735 sec.
ステップ( 8680 )  loss: 3.4038 -- time: 4.6219 sec.
ステップ( 8690 )  loss: 2.9972 -- time: 4.6025 sec.
ステップ( 8700 )  loss: 3.3221 -- time: 4.5387 sec.
ステップ( 8710 )  loss: 3.4139 -- time: 4.6200 sec.
ステップ( 8720 )  loss: 3.2163 -- time: 4.6784 sec.
ステップ( 8730 )  loss: 3.5807 -- time: 4.5700 sec.
ステップ( 8740 )  loss: 3.5082 -- time: 4.6163 sec.
ステップ( 8750 )  loss: 3.3746 -- time: 4.5308 sec.
ステップ( 8760 )  loss: 3.7508 -- time: 4.5185 sec.
ステップ( 8770 )  loss: 3.6274 -- time: 4.5865 sec.
--------------------------
train_loss: 618.5308 - val_loss: 0.0000
time: 84.8126 sec.
--------------------------
Epoch 50/50
-----------

/home/komuro/Documents/ssdbook/augmentation.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


ステップ( 8780 )  loss: 3.0061 -- time: 4.0536 sec.
ステップ( 8790 )  loss: 3.2130 -- time: 4.5674 sec.
ステップ( 8800 )  loss: 3.1602 -- time: 4.5560 sec.
ステップ( 8810 )  loss: 3.7752 -- time: 4.4853 sec.
ステップ( 8820 )  loss: 2.9803 -- time: 4.5440 sec.
ステップ( 8830 )  loss: 3.6303 -- time: 4.4915 sec.
ステップ( 8840 )  loss: 3.8253 -- time: 4.5299 sec.
ステップ( 8850 )  loss: 2.8480 -- time: 4.5682 sec.
ステップ( 8860 )  loss: 3.8794 -- time: 4.5535 sec.
ステップ( 8870 )  loss: 3.0537 -- time: 4.6792 sec.
ステップ( 8880 )  loss: 2.9956 -- time: 4.6507 sec.
ステップ( 8890 )  loss: 3.0756 -- time: 4.5316 sec.
ステップ( 8900 )  loss: 2.8362 -- time: 4.4318 sec.
ステップ( 8910 )  loss: 3.5260 -- time: 4.5330 sec.
ステップ( 8920 )  loss: 3.2934 -- time: 4.5755 sec.
ステップ( 8930 )  loss: 3.3709 -- time: 4.5573 sec.
ステップ( 8940 )  loss: 3.6246 -- time: 4.6817 sec.
ステップ( 8950 )  loss: 3.2931 -- time: 4.4537 sec.
--------------------------
validation
--------------------------
train_loss: 607.2673 - val_loss: 696.2233
time: 117.9840 sec.
--saved w